# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [4]:
data = pd.read_csv('/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/lab-intro-recommender-systems/data/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [6]:
total_q = pd.pivot_table(data, index=['CustomerID','ProductName'], values=['Quantity'], aggfunc='sum')
total_q.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [7]:
product_matrix = pd.pivot_table(data, index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
product_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
distances = pd.DataFrame(1/(1 + squareform(pdist(product_matrix.T, 'euclidean'))), index=product_matrix.columns, columns=product_matrix.columns)
distances.shape

(1000, 1000)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [17]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
3909    0.095358
264     0.093953
3531    0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [20]:
records = total_q.loc[similarities.index]
records

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[282 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [21]:
aggregation = records.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
aggregation

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3
...,...
Gatorade - Xfactor Berry,1
Gloves - Goldtouch Disposable,1
Grouper - Fresh,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [22]:
a = 0
for product in aggregation.index:
    if product_matrix.loc[product][33]==0:
        print(product)
        a += 1
    if a == 5:
        break

Chocolate - Dark
Wine - Charddonnay Errazuriz
Bay Leaf
Oranges - Navel, 72
Wine - Ej Gallo Sierra Valley


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [24]:
recomendations = {}
def recomendation(CustomerID):
    rec = []
    similarities = distances[CustomerID].sort_values(ascending=False)[1:6]
    records = total_q.loc[similarities.index]
    aggregation = pd.pivot_table(records, index=['ProductName'], values=['Quantity'],aggfunc='sum').sort_values(by='Quantity',ascending=False)
    a = 0
    for product in aggregation.index:
        if product_matrix.loc[product][CustomerID] == 0:
            rec.append(product)
            a += 1
        if a == 5:
            break
    recomendations[CustomerID] = rec

In [28]:
for client in product_matrix.columns:
    recomendation(client)

In [29]:
recomendations

'Lettuce - Frisee',
  'Tahini Paste',
  'Turnip - White, Organic',
  'Crackers - Trio'],
 84972: ['Sole - Dover, Whole, Fresh',
  'Beans - Kidney, Red Dry',
  'Hickory Smoke, Liquid',
  'Scallops - Live In Shell',
  'Muffin - Carrot Individual Wrap'],
 85032: ['Cheese Cloth No 100',
  'Water, Tap',
  'Bar Mix - Pina Colada, 355 Ml',
  'Sponge Cake Mix - Chocolate',
  'Cup - Translucent 7 Oz Clear'],
 85161: ['Bread - Roll, Canadian Dinner',
  'Chef Hat 20cm',
  'Fenngreek Seed',
  'Coffee Decaf Colombian',
  'Apricots - Halves'],
 85246: ['Pickerel - Fillets',
  'Chocolate - Compound Coating',
  'Pepper - Paprika, Hungarian',
  'Longos - Grilled Chicken With',
  'Appetizer - Sausage Rolls'],
 85292: ['Wine - Chablis 2003 Champs',
  'Soupfoamcont12oz 112con',
  'Wasabi Powder',
  'Pastry - Choclate Baked',
  'Beer - Blue'],
 85437: ['Wine - Ruffino Chianti',
  'Wine - Chardonnay South',
  'Sage - Ground',
  'Beef Ground Medium',
  'Cheese - Parmesan Grated'],
 85480: ['Salmon - Atlantic

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [31]:
recomendations = pd.DataFrame(recomendations)
recomendations

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Chocolate - Dark,Soup - Campbells Bean Medley,Cheese - Parmesan Grated,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Cheese - Mozzarella,Cheese - Parmesan Grated,Lamb - Ground,Knife Plastic - White,Cod - Black Whole Fillet,...,"Lentils - Red, Dry",Muffin Batt - Choc Chk,Sobe - Tropical Energy,"Lentils - Red, Dry",Cup - Translucent 7 Oz Clear,Beans - Kidney White,Skirt - 29 Foot,Butter - Unsalted,"Peas - Pigeon, Dry",Hot Chocolate - Individual
1,Wine - Charddonnay Errazuriz,Pomello,Pork - Kidney,"Pepper - White, Ground",Beef - Top Sirloin,Ice Cream Bar - Hageen Daz To,"Pepper - Black, Whole",Beef - Texas Style Burger,Wine - Toasted Head,Beans - Wax,...,Cheese - Wine,Wine - Chablis 2003 Champs,"Pork - Loin, Bone - In",Sausage - Liver,Rice - Jasmine Sented,"Appetizer - Mini Egg Roll, Shrimp",Beans - Kidney White,Halibut - Steaks,Squid U5 - Thailand,Cheese - Mozzarella
2,Bay Leaf,Bay Leaf,Crush - Cream Soda,Cup - Translucent 7 Oz Clear,Butter - Unsalted,Knife Plastic - White,Sugar - Fine,Soup - Campbells Tomato Ravioli,Ice Cream Bar - Hageen Daz To,"Pepper - Black, Whole",...,Cassis,French Pastry - Mini Chocolate,Cake - Box Window 10x10x2.5,Liners - Baking Cups,"Sole - Dover, Whole, Fresh",Pomello,Milk - 1%,"Wine - Red, Harrow Estates, Cab",Cheese - Taleggio D.o.p.,"Hickory Smoke, Liquid"
3,"Oranges - Navel, 72",Pork - Kidney,Soup - Campbells Tomato Ravioli,Sage - Ground,Table Cloth 81x81 White,"Peas - Pigeon, Dry",Olive - Spread Tapenade,"Veal - Brisket, Provimi,bnls",Pork - Kidney,Kellogs Special K Cereal,...,Sauce - Demi Glace,Muffin Chocolate Individual Wrap,Garlic - Elephant,Lettuce - Frisee,"Pork - Loin, Center Cut",Ecolab - Lime - A - Way 4/4 L,Cheese - Taleggio D.o.p.,Ice Cream Bar - Oreo Cone,"Cheese - Brie, Triple Creme",Garlic
4,Wine - Ej Gallo Sierra Valley,Muffin - Carrot Individual Wrap,"Pepper - White, Ground","Pork - Bacon, Double Smoked","Pepper - Black, Whole","Beets - Candy Cane, Organic",Quiche Assorted,Beets - Mini Golden,Juice - Apple Cider,Ecolab - Mikroklene 4/4 L,...,Flour - Pastry,"Cheese - Boursin, Garlic / Herbs",Appetizer - Mushroom Tart,Bandage - Flexible Neon,"Appetizer - Mini Egg Roll, Shrimp","Veal - Inside, Choice",Sprouts - Baby Pea Tendrils,Vinegar - Tarragon,Crackers - Trio,"Cheese - Brie, Triple Creme"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [32]:
distances = pd.DataFrame(1/(1 + squareform(pdist(product_matrix.T, 'correlation'))), index=product_matrix.columns, columns=product_matrix.columns)

for client in product_matrix.columns:
    recomendation(client)

recomendations = pd.DataFrame(recomendations)
recomendations

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,"Rosemary - Primerba, Paste",Pasta - Orecchiette,"Water - Mineral, Natural","Yogurt - Blueberry, 175 Gr",Muffin Mix - Blueberry,Turkey - Oven Roast Breast,Fenngreek Seed,"Pepper - White, Ground","Wine - White, Schroder And Schyl",Tuna - Salad Premix,...,Apricots Fresh,Wine - Fume Blanc Fetzer,Veal - Sweetbread,"Beef - Ground, Extra Lean, Fresh",Wine - White Cab Sauv.on,"Wine - Red, Cooking","Soup - Canadian Pea, Dry Mix",Curry Paste - Madras,Duck - Breast,Carbonated Water - Cherry
1,Cornflakes,Pasta - Cheese / Spinach Bauletti,Chicken - Wieners,"Water - Mineral, Natural",Sprouts - Alfalfa,Knife Plastic - White,Lettuce - Spring Mix,Pail With Metal Handle 16l White,Muffin Mix - Blueberry,Pail For Lid 1537,...,Juice - Apple Cider,Carbonated Water - Cherry,"Wine - White, Mosel Gold",Cassis,Dc - Frozen Momji,Papayas,"Mushroom - Trumpet, Dry",Cookie Chocolate Chip With,Cheese - Taleggio D.o.p.,Juice - Orange
2,Beef - Short Loin,General Purpose Trigger,Cookie - Dough Variety,Rum - Mount Gay Eclipes,Crackers - Trio,Meldea Green Tea Liquor,Vinegar - Sherry,Apricots - Dried,Cheese - Victor Et Berthold,Cod - Black Whole Fillet,...,Cumin - Whole,Cheese - Victor Et Berthold,Wine - Pinot Noir Latour,"Spoon - Soup, Plastic",Lime Cordial - Roses,Nantuket Peach Orange,Sobe - Tropical Energy,Flour - Whole Wheat,Cheese - Parmesan Cubes,Table Cloth 62x114 White
3,Juice - Apple Cider,Langers - Ruby Red Grapfruit,"Lamb - Whole, Fresh",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Bistro White,Veal - Inside,"Wine - Red, Harrow Estates, Cab","Salsify, Organic",Smirnoff Green Apple Twist,...,Squid U5 - Thailand,Rice - Jasmine Sented,Flour - Teff,Cake - Box Window 10x10x2.5,Beef - Striploin Aa,Lime Cordial - Roses,Beans - Kidney White,Banana - Leaves,Cookie Dough - Double,"Nut - Pistachio, Shelled"
4,Chinese Foods - Chicken,Apricots - Dried,Cake - Box Window 10x10x2.5,Cod - Black Whole Fillet,Pomello,Wine - Magnotta - Belpaese,Kiwi,"Oil - Shortening,liqud, Fry",Browning Caramel Glace,Beer - Rickards Red,...,Chef Hat 20cm,Blackberries,"Rum - Coconut, Malibu",Beets - Mini Golden,"Oil - Shortening,liqud, Fry","Yogurt - Blueberry, 175 Gr",Onions - Vidalia,Bandage - Fexible 1x3,Squid U5 - Thailand,Lime Cordial - Roses
